In [1]:
# Standard Python modules
import os, sys
import glob
import numpy as np
import pandas as pd
import xarray as xr
import re
import datetime

In [2]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [6]:
## Load tARgetv4 AR data
fname = path_to_data + 'downloads/globalARcatalog_ERA5_1940-2023_v4.0.nc'
ds = xr.open_dataset(fname)
ds = ds.isel(lev=0, ens=0)
ds = ds.assign_coords({"lon": (((ds.lon + 180) % 360) - 180)}) # Convert DataArray longitude coordinates from 0-359 to -180-179
latmin, latmax, lonmin, lonmax = 10., 70., -140., -80.
ds = ds.sel(lat=slice(latmax, latmin), lon=slice(lonmin, lonmax))
ds = ds.sel(time=slice('2000-01-01', '2023-12-31'))

ds

<xarray.Dataset>
Dimensions:      (lon: 241, lat: 241, time: 35064)
Coordinates:
  * lon          (lon) float64 -140.0 -139.8 -139.5 ... -80.5 -80.25 -80.0
  * lat          (lat) float64 70.0 69.75 69.5 69.25 ... 10.75 10.5 10.25 10.0
    lev          float64 0.0
  * time         (time) datetime64[ns] 2000-01-01 ... 2023-12-31T18:00:00
    ens          float64 1.0
Data variables: (12/55)
    islnd        (lat, lon) float32 ...
    iscst        (lat, lon) float32 ...
    ivt_begin    (lat, lon) float64 ...
    ivt_end      (lat, lon) float64 ...
    shapemap     (time, lat, lon) float32 ...
    axismap      (time, lat, lon) float64 ...
    ...           ...
    kinstveldir  (time, lat) float32 ...
    axisidx      (time, lat, lon) float32 ...
    axislon      (time, lat, lon) float32 ...
    axislat      (time, lat, lon) float32 ...
    numobj       (time, lat, lon) float32 ...
    intblosto    (lat, lon) float64 ...
Attributes:
    title:                 Global Atmospheric River Database
    version:               2024.03.30
    creation_date:         30-Mar-2024 13:15:27
    tARget_version:        1.312
    dimension_convention:  5-d (nlon x nlat x nlev x ntime x nens) variables ...
    references:            (1) Guan, B., and D. E. Waliser (2015), Detection ...

In [26]:
## CODE TO PULL THE LATLON COORD PAIRS FOR THE COAST
new_da = ds.iscst.where(ds.iscst==True) ## get coastal values
idx_lst = np.argwhere(new_da.values==1) ## now find index where coastal values are true
idx_lst

lat_lst = []
lon_lst = []
for i, idx in enumerate(idx_lst):
    tmp = ds.iscst.isel(lat=idx[0], lon=idx[1]) # subset to current idx
    lat_lst.append(tmp.lat.values)
    lon_lst.append(tmp.lon.values)


d = {'lat' : lat_lst,
        'lon' : lon_lst}

txtpts = pd.DataFrame(d)

txtpts.to_csv('../out/latlon_coast_ERA5.csv', index=False) 

In [10]:
## save file
ds = ds.kidmap # subset to kidmap da
fname = path_to_data + 'preprocessed/tARgetv4/globalARcatalog_ERA5_2000-2023_v4.0.nc'
ds.to_netcdf(path=fname, mode = 'w', format='NETCDF4')